In [1]:
# Importando o Pandas

import pandas as pd

In [2]:
# Importando base de dados

tabela = pd.read_csv(r'C:\Users\Lucy Minene\Desktop\tratamento-variaveis-texto\Clean_Dataset.csv', index_col=[0])

tabela.head(3)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956


In [3]:
# Traduzindo os nomes das colunas

tabela.columns = ['empresa_area', 'voo', 'origem', 'hr_embarque', 
                  'paradas', 'hr_chegada', 'destino', 'classe', 'duracao', 'dias_antes', 'preco']

In [4]:
# Verificando as informações do DF
# Colunas 0 a 7 estão todas no formato de texto

tabela.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300153 entries, 0 to 300152
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   empresa_area  300153 non-null  object 
 1   voo           300153 non-null  object 
 2   origem        300153 non-null  object 
 3   hr_embarque   300153 non-null  object 
 4   paradas       300153 non-null  object 
 5   hr_chegada    300153 non-null  object 
 6   destino       300153 non-null  object 
 7   classe        300153 non-null  object 
 8   duracao       300153 non-null  float64
 9   dias_antes    300153 non-null  int64  
 10  preco         300153 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 27.5+ MB


## Eliminando coluna desnecessária

In [5]:
#Verificando a cardinalidade do Data Frame

tabela.nunique()

empresa_area        6
voo              1561
origem              6
hr_embarque         6
paradas             3
hr_chegada          6
destino             6
classe              2
duracao           476
dias_antes         49
preco           12157
dtype: int64

In [6]:
# Eliminando a coluna 'voo'

tabela = tabela.drop('voo', axis=1)

##  Utilizando apply e lambda function

In [7]:
# Visualizando os valores da coluna 'class'

tabela.classe.value_counts()

Economy     206666
Business     93487
Name: classe, dtype: int64

#### Como exitem apenas 2 tipos de classe, economica e executiva, irei criar uma coluna com o valor 1 para classe economica e valor 0 para classe executiva.

In [8]:
# Criando a nova coluna

tabela['economica'] = tabela.classe.apply(lambda x:1 if x == 'Economy' else 0)

In [9]:
# Visualizando valores

tabela[['classe', 'economica']].value_counts()

classe    economica
Economy   1            206666
Business  0             93487
dtype: int64

### Criando uma função para tratamento dos dados

In [10]:
# Visualizando os valores da coluna "paradas"

tabela.paradas.value_counts()

one            250863
zero            36004
two_or_more     13286
Name: paradas, dtype: int64

In [11]:
# Criando uma função para tratar os valores da coluna "paradas"

def num_paradas(valor):
    if valor == 'zero':
        return 0
    elif valor == 'one':
        return 1
    elif valor == 'two_or_more':
        return 2
    else:
        return -1

In [12]:
# Aplicando a função

tabela['num_paradas'] = tabela.paradas.apply(num_paradas)

In [13]:
# Visualizando valores

tabela[['paradas', 'num_paradas']].value_counts()

paradas      num_paradas
one          1              250863
zero         0               36004
two_or_more  2               13286
dtype: int64

### Utilizando OneHotEncoder

In [14]:
# Verificando a coluna "empresa_área"

tabela.empresa_area.value_counts()

Vistara      127859
Air_India     80892
Indigo        43120
GO_FIRST      23173
AirAsia       16098
SpiceJet       9011
Name: empresa_area, dtype: int64

In [15]:
# Criando o codificador

from sklearn.preprocessing import OneHotEncoder

ohe_empresa_area = OneHotEncoder(handle_unknown='ignore')

# Fazendo fit com os dados

ohe_empresa_area = ohe_empresa_area.fit(tabela[['empresa_area']])

# Fazendo a transformação dos dados

ohe_empresa_area.transform(tabela[['empresa_area']])

<300153x6 sparse matrix of type '<class 'numpy.float64'>'
	with 300153 stored elements in Compressed Sparse Row format>

In [16]:
# Visualizando no formato de array

ohe_empresa_area.transform(tabela[['empresa_area']]).toarray()

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [17]:
# Transformando o array em DataFrame

ohe_df = pd.DataFrame(ohe_empresa_area.transform(tabela[['empresa_area']]).toarray())

ohe_df.head(5)

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
# Selencionando o nome das colunas

colunas = ohe_empresa_area.get_feature_names_out()

colunas

array(['empresa_area_AirAsia', 'empresa_area_Air_India',
       'empresa_area_GO_FIRST', 'empresa_area_Indigo',
       'empresa_area_SpiceJet', 'empresa_area_Vistara'], dtype=object)

In [19]:
# Utilizando get_feature_names_out para nomear as colunas

ohe_df = pd.DataFrame(ohe_empresa_area.transform(tabela[['empresa_area']]).toarray(),
                     columns = colunas,
                      dtype = 'int32'
                     )

ohe_df.head(5)

,empresa_area_AirAsia,empresa_area_Air_India,empresa_area_GO_FIRST,empresa_area_Indigo,empresa_area_SpiceJet,empresa_area_Vistara
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,1,0,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [20]:
# Unir o df acima com o original

tabela = pd.concat([tabela, ohe_df], axis=1)

tabela.head(5)

,empresa_area,origem,hr_embarque,paradas,hr_chegada,destino,classe,duracao,dias_antes,preco,economica,num_paradas,empresa_area_AirAsia,empresa_area_Air_India,empresa_area_GO_FIRST,empresa_area_Indigo,empresa_area_SpiceJet,empresa_area_Vistara
0,SpiceJet,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953,1,0,0,0,0,0,1,0
1,SpiceJet,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953,1,0,0,0,0,0,1,0
2,AirAsia,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956,1,0,1,0,0,0,0,0
3,Vistara,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955,1,0,0,0,0,0,0,1
4,Vistara,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955,1,0,0,0,0,0,0,1


In [21]:
# Resultado

tabela.groupby('empresa_area')[colunas].sum()

,empresa_area_AirAsia,empresa_area_Air_India,empresa_area_GO_FIRST,empresa_area_Indigo,empresa_area_SpiceJet,empresa_area_Vistara
empresa_area,,,,,,
AirAsia,16098,0,0,0,0,0
Air_India,0,80892,0,0,0,0
GO_FIRST,0,0,23173,0,0,0
Indigo,0,0,0,43120,0,0
SpiceJet,0,0,0,0,9011,0
Vistara,0,0,0,0,0,127859
